In [1]:
import math
import shutil
import torch
import gpytorch
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import os
import h5py
import sys
current_dir = "/work/07059/jyzhao/ls6/cybershake_ngmm/Scalable_GPs_NGMM/regression_gpytorch/src"
if current_dir not in sys.path:
    sys.path.append(current_dir)
from training_utils.GPModel import ExactGPModel
from training_utils.utils import set_seed, read_hdf5, predict_dataset_pre_select
from training_utils.utils import str2bool
import time
from torch.utils.data import TensorDataset, DataLoader
import argparse
from training_utils.predict_utils import load_ExactGP_model

In [2]:
model_path = "/work/07059/jyzhao/ls6/cybershake_ngmm/Scalable_GPs_NGMM/regression_gpytorch/output/trained_models_exact/4179_training_eqs_250_epochs_no_source_train_on_mean/all_rlzs_v1_set_initial_hyperparameters/gpu_model_trained.pth"
likelihood_path = "/work/07059/jyzhao/ls6/cybershake_ngmm/Scalable_GPs_NGMM/regression_gpytorch/output/trained_models_exact/4179_training_eqs_250_epochs_no_source_train_on_mean/all_rlzs_v1_set_initial_hyperparameters/gpu_likelihood_trained.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_x = torch.zeros(1, 9).to(device) # placeholder
train_y = torch.zeros(1,).to(device) # placeholder
model, likelihood = load_ExactGP_model(model_path, likelihood_path,
                                               train_x=train_x, train_y=train_y, device=device)

In [19]:
# Train on mean
# loss_path = "/work/07059/jyzhao/ls6/cybershake_ngmm/Scalable_GPs_NGMM/regression_gpytorch/output/trained_models_exact/4179_training_eqs_250_epochs_no_source_train_on_mean/all_rlzs_v1_set_initial_hyperparameters/training_hyperparameter_loss.npy"
# Train on first realization
loss_path = "/work/07059/jyzhao/ls6/cybershake_ngmm/regression_gpytorch/output/trained_models_exact/4179_training_eqs_500_epochs_no_source/training_hyperparameter_loss.npy"
training_loss = np.load(loss_path)

In [20]:
fig, axes = plt.subplots(1,3,figsize=(16, 4))
axes[0].plot(training_loss[:,1], label='Site_lengthscale')
axes[0].plot(training_loss[:,3], label='Path_lengthscale')

axes[1].plot(training_loss[:,0], label='Site_variance')
axes[1].plot(training_loss[:,2], label='Path_variance')
axes[1].plot(training_loss[:,4], label='likelihood_noise')
axes[1].plot(training_loss[:,6], label='Between event variance')

axes[2].plot(training_loss[:,5], label='LOO loss')

axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Hyperparameter Value')
axes[0].legend()

axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Hyperparameter Value')
axes[1].legend()

axes[2].set_ylim([0,2])
axes[2].set_xlabel('Iteration')
axes[2].set_ylabel('Training loss')
axes[2].legend()

fig.savefig(os.path.join(current_dir, 'training_loss_parameters.png'), dpi=300)
plt.close(fig)